In [7]:
import numpy as np
import pandas as pd
import time, json
from datetime import date
from statsmodels.tsa.stattools import adfuller, acf, pacf
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import mean_squared_error


#Les données à entrer.
#La mise initiale en usdt
money = 1000
#Prix du trading.
fees = 0.0005
#Pair à trader (pour l'instant on ne prend que des pairs avec USDT et on considère que USDT = USD).
pair = "USDT_ETH"
#Date début en timestamp
start = "1483228800"
#Date fin en timestamp
end = "1521384257"
#Période par trading en secondes
period = "300"

print('On charge data')
df = pd.read_json("https://poloniex.com/public?command=returnChartData&currencyPair="+pair+"&start="+start+"&end="+end+"&period="+period,convert_dates=['date'])

#Mettre la date en index, pandas comprend que c'est une timeseries.
df=df.set_index('date')


wa_data = df['weightedAverage']
#size permet de préciser le nombre de valeurs à prédire, remplacer 2 par le nombre de valeurs
size = int(len(wa_data) - 2)
#On crée la liste des prix d'ouverture (changer open par close, high, low, pour les différentes prédictions).
prices = np.array(df["weightedAverage"])[size:len(wa_data)]

#Ici on doit mettre la liste des prédictions
wa_data = df['weightedAverage']
size = int(len(wa_data) - 2)
train, test = wa_data[0:size], wa_data[size:len(wa_data)]
history = [x for x in train]
predictions = list()

print('Printing Predicted')
print('\n')
for t in range(len(test)):
    model = ARIMA(history, order=(2,1,1))
    model_fit = model.fit(disp=0)
    output = model_fit.forecast()
    yhat = output[0]
    predictions.append(float(yhat))
    obs = test[t]
    history.append(obs)
    print('predicted=%f, expected=%f, day=%s, diff=%f' % (yhat, obs,str(test.index[t]),(yhat-obs)*100/yhat))

error = mean_squared_error(test, predictions)
predictions = np.array(predictions)

#On calcul l'argent final de l'investisseur. C'est ça qu'on doit battre.
hodl_money = money*(1-fees)*prices[-1]/prices[0]

print("L'investisseur aura %d dollars à la fin de la période." % hodl_money)

#On utilise les prédictions pour acheter chaque fois qu'on pense gagner plus que plus% et vendre quand on pense perdre plus que minus%.
#C'est une méthode assez basique et on pourra faire mieux après.
plus = 5.
minus = 5.
#Argent du début et est ce qu'on a vendu ou pas.
trade_crypto = money*(1-fees)/prices[0]
trade_money = 0
sold = False
for i in range(len(prices)-1):
	if (predictions[i+1] > (1+plus/100)*prices[i] and sold == True):
		sold = False
		trade_crypto = trade_money*(1-fees)/prices[i]
		trade_money = 0
	elif (predictions[i+1] < (1-minus/100)*prices[i] and sold == False):
		sold = True
		trade_money = trade_crypto*prices[i]*(1-fees)
		trade_crypto = 0

#argent final du trading.
final_money = trade_money + trade_crypto*(1-fees)/prices[-1]

print("Le bot aura %d dollars à la fin de la période." % final_money)


On charge data
Printing Predicted


predicted=458.525782, expected=461.709159, day=2018-03-18 14:35:00, diff=-0.694263
predicted=462.367726, expected=468.803004, day=2018-03-18 14:40:00, diff=-1.391810
L'investisseur aura 1014 dollars à la fin de la période.
Le bot aura 0 dollars à la fin de la période.


In [6]:
list(range(len(prices)-1))

[0, 1, 2, 3, 4, 5, 6, 7, 8]